In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

import re
import time

rm_parenthesis = re.compile(r"\（.*?\）|\（ .*? \）|\(.*?\)|\( .*? \)")

## Получение датасета с пиньинем

In [2]:
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)

try:
    driver.get("https://mandarinbean.com/new-hsk-1-word-list/")
    headers = [header.get_attribute("innerHTML") for header in driver.find_elements(By.TAG_NAME, "strong")]
    final_data = pd.DataFrame(columns=headers)
    temp_data = [value.get_attribute("innerHTML") for value in driver.find_elements(By.TAG_NAME, "td")]

except Exception as e:
    print(f"{e}\n\nThe process will terminate in 10 seconds...")
    time.sleep(10)

finally:
    driver.close()
    driver.quit()

In [52]:
m_final_data = final_data
m_temp_data = temp_data

In [54]:
def divide_rows(row):   # Разделяет один ряд с n-ным количеством "|" на n+1 рядов
    result = []
    while "｜" in "".join(row):
        irow = [row[0], row[1][:row[1].index("｜")], row[2][:row[2].index("｜")-1], row[3]]
        row = [row[0], row[1][row[1].index("｜")+1:], row[2][row[2].index("｜")+2:], row[3]]
        result.append(irow)
    result.append([row[0], row[1][0:], row[2][0:], row[3]])
    return result 

data = []
for i in range(4, len(m_temp_data), 4):
    temp_arr = [re.sub(rm_parenthesis, "", td.replace("&nbsp;", " ")) for td in m_temp_data[i-4:i]]     # регулярку надо в другом месте применять
    if "｜" in "".join(temp_arr):
        temp_arr = divide_rows(temp_arr)
    data.extend([temp_arr] if len(temp_arr)==4 else temp_arr)

m_final_data = pd.DataFrame(data=data, columns=headers)
m_final_data.drop(labels=["No", "English"], axis=1, inplace=True)
m_final_data.head()
m_final_data.to_csv(path_or_buf=".\\test.csv", index=False)

## Скрейпинг для других систем транскрипции

In [17]:
final_data = pd.read_csv("./test.csv")

driver = webdriver.Chrome()

try:
    def get_palladius(url, src_table, variations=None):

        def clear_text(element):
            length = len(element.get_attribute('value'))
            element.send_keys(length * Keys.BACKSPACE)

        if variations is not None:
            pass

        driver.get(url)
        scraped = []
        element = WebDriverWait(driver, 10).until( 
            EC.presence_of_element_located((By.CLASS_NAME, "pinyin"))
        )

        for tra in src_table:
            textbox = driver.find_element(By.CLASS_NAME, "pinyin") 
            convert_button = driver.find_element(By.CLASS_NAME, "translate")

            textbox.send_keys(tra)
            convert_button.click()
    
            driver.implicitly_wait(1)
            scraped.append(driver.find_element(By.CLASS_NAME, "palladius").get_attribute("innerHTML"))
            clear_text(textbox)
        return scraped
    
    palladius = pd.Series(data=get_palladius("https://palladius.ru/", [tra for tra in final_data["Pinyin"]]))
    final_data = pd.concat(objs=[final_data, palladius], axis=1, index=False)
    print(final_data)

except Exception as e:
    print(f"{e}\n\nThe process will terminate in 10 seconds...")
    time.sleep(10)

finally:
    driver.close()
    driver.quit()

Message: element not interactable
  (Session info: chrome=119.0.6045.200)
Stacktrace:
	GetHandleVerifier [0x00007FF7AFA782B2+55298]
	(No symbol) [0x00007FF7AF9E5E02]
	(No symbol) [0x00007FF7AF8A045D]
	(No symbol) [0x00007FF7AF8DCA37]
	(No symbol) [0x00007FF7AF8DBD82]
	(No symbol) [0x00007FF7AF9020AA]
	(No symbol) [0x00007FF7AF8D80CF]
	(No symbol) [0x00007FF7AF9022C0]
	(No symbol) [0x00007FF7AF91AAA4]
	(No symbol) [0x00007FF7AF901E83]
	(No symbol) [0x00007FF7AF8D670A]
	(No symbol) [0x00007FF7AF8D7964]
	GetHandleVerifier [0x00007FF7AFDF0AAB+3694587]
	GetHandleVerifier [0x00007FF7AFE4728E+4048862]
	GetHandleVerifier [0x00007FF7AFE3F173+4015811]
	GetHandleVerifier [0x00007FF7AFB147D6+695590]
	(No symbol) [0x00007FF7AF9F0CE8]
	(No symbol) [0x00007FF7AF9ECF34]
	(No symbol) [0x00007FF7AF9ED062]
	(No symbol) [0x00007FF7AF9DD3A3]
	BaseThreadInitThunk [0x00007FFDA0407344+20]
	RtlUserThreadStart [0x00007FFDA21226B1+33]


The process will terminate in 10 seconds...


In [ ]:
driver.get("https://ctext.org/pinyin.pl?if=en")
element = WebDriverWait(driver, 10).until( 
    EC.presence_of_element_located((By.NAME, "text"))
)

textbox = driver.find_element(By.NAME, "text")
system_switch = driver.find_element(By.NAME, "to")
convert_button = driver.find_element(By.CSS_SELECTOR, "[value='Convert']") 

system_switch.

textbox.send_keys(final_data["Pinyin"][1])
convert_button.click()

## Далее клетки для полномасштабной работы
Не трогать пока не отлажен скрейпинг для HSK 1

In [30]:
options = ChromeOptions()
options.add_argument("--headless=new")
driver = webdriver.Chrome(options=options)

try:
    driver.get("https://mandarinbean.com/new-hsk-1-word-list/")
    headers = [header.get_attribute("innerHTML") for header in driver.find_elements(By.TAG_NAME, "strong")]
    final_data = pd.DataFrame(columns=headers)

    def get_table(url):     # Берёт страницу и собирает с неё все элементы таблицы (в тегах <td>) в датафрейм

        def divide_rows(row):   # Разделяет один ряд с n-ным количеством "|" на n+1 рядов
            result = []
            while "｜" in "".join(row):
                irow = [row[0], row[1][:row[1].index("｜")], row[2][:row[2].index("｜")-1], row[3]]
                row = [row[0], row[1][row[1].index("｜")+1:], row[2][row[2].index("｜")+2:], row[3]]
                result.append(irow)
            result.append([row[0], row[1][0:], row[2][0:], row[3]])
            return result 

        driver.get(url)
        element = WebDriverWait(driver, 10).until(          # Ожидание прогрузки элементов в тегах <td>
            EC.presence_of_element_located((By.TAG_NAME, "td"))
        )

        temp_data = [value.get_attribute("innerHTML") for value in driver.find_elements(By.TAG_NAME, "td")]

        data = []
        for i in range(4, len(temp_data), 4):
            temp_arr = [re.sub(rm_parenthesis, "", td.replace("&nbsp;", " ")) for td in temp_data[i-4:i]]
            if "｜" in "".join(temp_arr):
                temp_arr = divide_rows(temp_arr)
            data.extend([temp_arr] if len(temp_arr)==4 else temp_arr)

        return pd.DataFrame(data=data, columns=headers)

    for i in range(1, 7):
        final_data = pd.concat(objs=[final_data, get_table(f"https://mandarinbean.com/new-hsk-{i}-word-list/")]) 
        
    final_data.drop(labels=["No", "English"], axis=1, inplace=True)
    final_data.to_csv(path_or_buf=".\\pinyin.csv", index=False)

except Exception as e:
    print(f"{e}\n\nThe process will terminate in 10 seconds...")
    time.sleep(10)

finally:
    driver.close()
    driver.quit()

Shape of passed values is (1996, 1), indices imply (1996, 4)

The process will terminate in 10 seconds...


In [24]:
data = pd.read_csv("./pinyin.csv")
data.head()

,Chinese,Pinyin,English
0,爱,ài,love
1,爱好,ài hào,hobby
2,八,bā,eight
3,爸爸｜爸,bàba ｜ bà,dad
4,吧,ba,(interjection particle)
